In [ ]:
from typing import Any
from sgqlc.operation import Operation
from graphql_resources.my_schema import Query
from graphql_resources import GraphQLClient 
from pprint import pprint
from sgqlc.endpoint.http import HTTPEndpoint
from typing import Dict
from config import eldenring_api_host

In [ ]:
client = GraphQLClient(eldenring_api_host)


def compute_operation(
    client: GraphQLClient, root: str, args: dict[str, Any] = None, selection: Dict[str, Any] = None
):
    op = Operation(Query)
    root_field = getattr(op, root)()
    if selection is {}:
        for field in root_field.__dict__:
            getattr(root_field, field)
    apply_selection(root_field, selection or {})
    result = client.execute_query(op)
    return result


def apply_selection(field_obj, selection: dict):
    for name, sub in selection.items():
        child = getattr(field_obj, name)()
        if isinstance(sub, dict):
            apply_selection(child, sub)

result = compute_operation(
    client,
    "weapon",
)
print(result)

{'success': True, 'data': {'weapon': [{'id': '17f69c35d2cl0i1oh7zuqfb3mdvsj', 'name': 'Hand Axe', 'image': 'https://eldenring.fanapis.com/images/weapons/17f69c35d2cl0i1oh7zuqfb3mdvsj.png', 'description': 'Commonly known as a hatchet, this smaller variety of axe is an everyday work tool. Despite its short reach, it is easy to handle, exceling in successive attacks.', 'attack': [{'amount': 113, 'name': 'Phy'}, {'amount': 0, 'name': 'Mag'}, {'amount': 0, 'name': 'Fire'}, {'amount': 0, 'name': 'Ligt'}, {'amount': 0, 'name': 'Holy'}, {'amount': 100, 'name': 'Crit'}], 'defence': [{'amount': 42, 'name': 'Phy'}, {'amount': 28, 'name': 'Mag'}, {'amount': 28, 'name': 'Fire'}, {'amount': 28, 'name': 'Ligt'}, {'amount': 28, 'name': 'Holy'}, {'amount': 28, 'name': 'Boost'}], 'scalesWith': [{'scaling': 'D', 'name': 'Str'}, {'scaling': 'D', 'name': 'Dex'}], 'requiredAttributes': [{'amount': 9, 'name': 'Str'}, {'amount': 8, 'name': 'Dex'}], 'category': 'Axe', 'weight': 3.5}, {'id': '17f69667224l0i1oha

In [ ]:
def _coerce_number(x: Any) -> Any:
    if isinstance(x, (int, float)):
        return x
    if isinstance(x, str):
        try:
            return int(x)
        except ValueError:
            try:
                return float(x)
            except ValueError:
                return x
    return x

def compare(value: Any, op: str, expected: Any) -> bool:
    if op == "eq":
        return value == expected
    if op == "ne":
        return value != expected

    v_num = _coerce_number(value)
    e_num = _coerce_number(expected)

    if op == "gt":
        return v_num > e_num
    if op == "gte":
        return v_num >= e_num
    if op == "lt":
        return v_num < e_num
    if op == "lte":
        return v_num <= e_num

    if op == "contains":
        return expected in value if value is not None else False
    if op == "in":
        return value in expected

    raise ValueError(f"Unknown operator: {op}")

In [131]:
filter = {
    "name": ("eq", "Hand Axe"),
    "attack": {
        "amount": ("gt", 10), 
        "name": ("eq", "Phy")
    },
    "attack": {
        "amount": ("gt", 10), 
        "name": ("eq", "Crit")
    }
}
items = result['data']["weapon"]


def matches_filter(obj: Any, filt: dict) -> bool:
    if obj is None:
        return False
    for field, condition in filt.items():
        value = obj.get(field) if isinstance(obj, dict) else getattr(obj, field, None)

        if isinstance(condition, tuple):
            op, expected = condition
            if not compare(value, op, expected):
                return False

        elif isinstance(condition, dict):
            if isinstance(value, list):
                if not any(matches_filter(v, condition) for v in value):
                    return False
            elif isinstance(value, dict):
                if not matches_filter(value, condition):
                    return False
            else:
                return False

        else:
            if value != condition:
                return False
    return True

filtered_result = [s for s in items if matches_filter(s, filter)]
print(filtered_result)
            











[{'name': 'Hand Axe', 'attack': [{'amount': 113, 'name': 'Phy'}, {'amount': 0, 'name': 'Mag'}, {'amount': 0, 'name': 'Fire'}, {'amount': 0, 'name': 'Ligt'}, {'amount': 0, 'name': 'Holy'}, {'amount': 100, 'name': 'Crit'}]}]
